In [35]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from math import sqrt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [2]:
def json_to_prices(json):
    df = pd.read_json(json)
    prices = df['prices']
    coin_prices = pd.DataFrame(prices.tolist(), columns=['time', 'price'])
    prices = coin_prices['price']
    return prices

In [21]:
def normalize_prices(prices):
    normalized_prices = (prices - np.mean(prices)) / np.std(prices)
    return normalized_prices

def split_data(normalized_prices, split=0.8):
    train_size = int(len(normalized_prices) * split)
    test_size = len(normalized_prices) - train_size
    
    training_data = normalized_prices[0:train_size]
    test_data = normalized_prices[train_size:len(normalized_prices)]
    return training_data, test_data

def create_dataset_helper(data, time_step=1):
    X, y = [], []
    # data = data.reset_index(drop=True)

    for i in range(len(data) - time_step):      
        X.append(data[i:(i + time_step)])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

def create_dataset(data, time_steps=1):
    X, y = create_dataset_helper(data, time_steps)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i])
        y.append(data[i, 0])  # Assuming the first column is DAI price
    return np.array(X), np.array(y)

In [29]:
def create_model(features, time_step):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, features)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# def create_multivariate_model():
#     model = Sequential()
#     model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 3)))
#     model.add(LSTM(units=50))
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model

def predictions(model, X_train, y_train, X_test, y_test, prices):
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    train_predict = (train_predictions * np.std(prices)) + np.mean(prices)
    y_train = (y_train * np.std(prices)) + np.mean(prices)
    test_predict = (test_predictions * np.std(prices)) + np.mean(prices)
    y_test = (y_test * np.std(prices)) + np.mean(prices)
    
    train_score = sqrt(mean_squared_error(y_train, train_predict))
    print('Train Score: %.20f RMSE' % (train_score))
    test_score = sqrt(mean_squared_error(y_test, test_predict))
    print('Test Score: %.20f RMSE' % (test_score))
    return train_predict, test_predict
    

In [22]:
# Load the data

dai = json_to_prices('dai90days.json')
eth = json_to_prices('ethereum90days.json')
usdc = json_to_prices('usdc90days.json')

#normalize data

normalized_dai = normalize_prices(dai)
normalized_eth = normalize_prices(eth)
normalized_usdc = normalize_prices(usdc)


In [23]:
# combine the data

combined_data = pd.concat([normalized_dai, normalized_eth, normalized_usdc], axis=1)

In [24]:
# remove null values

combined_data = combined_data.dropna()

# remove column names
combined_data = combined_data.values
print(combined_data)

[[ 1.36337107 -1.50989476  0.31397376]
 [ 0.02828203 -1.49104902 -0.13996421]
 [ 0.67319998 -1.50721905 -0.70738667]
 ...
 [ 1.34129665  3.23881176  0.54094275]
 [ 2.75704907  3.19242724 -1.50177812]
 [ 0.69631532  3.21266332  0.08700478]]


In [25]:
#split the data
time_step = 10

combined_training_data, combined_test_data = split_data(combined_data)

X_train, y_train = create_sequences(combined_training_data, time_step)
X_test, y_test = create_sequences(combined_test_data, time_step)

In [26]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1718, 10, 3) (1718,) (422, 10, 3) (422,)


In [30]:
model = create_model(3, time_step)
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
27/27 [==============================] - 3s 8ms/step - loss: 0.8754
Epoch 2/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8457
Epoch 3/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8335
Epoch 4/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8309
Epoch 5/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8202
Epoch 6/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8172
Epoch 7/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8153
Epoch 8/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8068
Epoch 9/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7997
Epoch 10/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7970
Epoch 11/50
27/27 [==============================] - 0s 7ms/step - loss: 0.7944
Epoch 12/50
27/27 [==============================] - 0s 7ms/step - loss: 0.8027
Epoch 13/50
27/27 [==============================

In [36]:
train_predict, test_predict = predictions(model, X_train, y_train, X_test, y_test, dai_data)

14/14 [==============================] - 0s 2ms/step
Train Score: 0.00093112595851800324 RMSE
Test Score: 0.00151115447449825949 RMSE
